In [3]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [4]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [5]:
tokenize('$5000')

['$', '5000']

In [6]:
def stem(word):
    return stemmer.stem(word.lower())

In [7]:
stem('organic')

'organ'

In [8]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [9]:
bag_of_words(['how','are','fwafaw','you'],['you','no','no','bro','how'])

array([1., 0., 0., 0., 1.], dtype=float32)

In [10]:
data = pd.read_csv('./tweets.csv').dropna()[:5000]

In [11]:
X = data['text']
y = data['target']

In [12]:
all_words = []
all_data = []
tags = []

In [13]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

5000it [00:01, 2801.99it/s]


In [14]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [15]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [16]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [17]:
X = []
y = []

In [18]:
for sentence,tag in tqdm(all_data):
    bag = bag_of_words(sentence,all_words)
    X.append(bag)
    y.append(tags.index(tag))

100%|██████████████████████████████████████| 5000/5000 [00:13<00:00, 367.00it/s]


In [19]:
X[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [20]:
y[0]

1

In [21]:
from sklearn.model_selection import *

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [23]:
device = 'cuda:0'

In [24]:
print('-')
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
print('-')
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
print('-')
y_train = torch.from_numpy(np.array(y_train)).to(device)
print('-')
y_test = torch.from_numpy(np.array(y_test)).to(device)
print('-')

In [25]:
# torch.save(X_train,'X_train.pth')
# torch.save(X_test,'X_test.pth')
# torch.save(y_train,'y_train.pth')
# torch.save(y_test,'y_test.pth')
# torch.save(all_data,'all_data.pth')
# torch.save(all_words,'all_words.pth')
# torch.save(tags,'tags.pth')
X_train = torch.load('X_train.pth').to(device).float()
y_train = torch.load('y_train.pth').to(device).float()
X_test = torch.load('X_test.pth').to(device).float()
y_test = torch.load('y_test.pth').to(device).float()
all_data = list(torch.load('all_data.pth'))
all_words = list(torch.load('all_words.pth'))
tags = list(torch.load('tags.pth'))

AttributeError: 'list' object has no attribute 'to'